# Aula prática: Destinação de Produtos à Fabricas
<sup>Exemplo do capítulo 8.3 do livro `Introdução à Pesquisa Operacional` de `Hillier e Lieberman`.</sup>

## Exercício 1

### Descrição do problema
A Cia. Produtos Melhores decidiu iniciar a produção de quatro produtos novos usando três fábricas que, no momento, têm excesso de capacidade produtiva. Os produtos requerem um esforço de produção comparável por unidade, de modo que a capacidade produtiva disponível das fábricas seja medida pelo número de unidades de qualquer produto que possa ser produzido diariamente, conforme dado na tabela 1. A tabela 2 fornece a taxa de produção diária necessária para atender às vendas projetadas. Cada fábrica é capaz de produzir qualquer um desses produtos, exceto a Fábrica 2, que não pode fabricar o produto 3. Os custos variáveis por unidade de cada produto diferem de fábrica para fábrica, conforme a tabela 3.

Tabela 1: Capacidade de produção disponível, por unidade do produto

| | Fábrica 1 | Fábrica 2 | Fábrica 3 |
|:---|:---:|:---:|:---:|
| Capacidade disponível | 75 | 75 | 45 |

Tabela 2: Demanda de produção diária

| | Produto 1 | Produto 2 | Produto 3 | Produto 4 |
|:---|:---:|:---:|:---:|:---:|
| Demanda | 20 | 30 | 30 | 40 |

Tabela 3: Custo unitário por produto

| | Produto 1 | Produto 2 | Produto 3 | Produto 4 |
|:---|:---:|:---:|:---:|:---:|
| Fábrica 1 | 41 | 27 | 28 | 24 |
| Fábrica 2 | 40 | 29 | - | 23 |
| Fábrica 3 | 37 | 30 | 27 | 21 |

A gerência precisa tomar uma decisão sobre como dividir a fabricação dos produtos entre as fábricas, e decidiu permitir a divisão da produção de um mesmo produto em mais de uma fábrica.

Escreva um modelo para determinar quais fábricas produzirão quais produtos e a que custo total.

### Resolução

In [2]:
# instalação e importação do pacote mip
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 51.7 MB/s eta 0:00:00


##Manual

In [60]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value, LpStatus

# Criação do modelo
model = LpProblem(name="minimization_problem", sense=LpMinimize)

# Adicionar variáveis
for i in range(1,4):
  for j in range(1,5):
    model += LpVariable(f"x{i}{j}", lowBound=0, cat='Integer')

# Definir a função objetivo
model += (41*x11 + 27*x12 + 28*x13 + 24*x14 +
          40*x21 + 29*x22 + 23*x24 +
          37*x31 + 30*x32 + 27*x33 + 21*x34), "Objective"

# Adicionar restrições
model += (x11 + x12 + x13 + x14 <= 75), "Constraint1"
model += (x21 + x22 + x24 <= 75), "Constraint2"
model += (x31 + x32 + x33 + x34 <= 45), "Constraint3"

model += (x11 + x21 + x31 >= 20), "Constraint4"
model += (x12 + x22 + x32 >= 30), "Constraint5"
model += (x13 + x33 >= 30), "Constraint6"
model += (x14 + x24 + x34 >= 40), "Constraint7"

# Resolver o modelo
model.solve()

# Verificar e imprimir resultados
print("Status:", LpStatus[model.status])
for var in model.variables():
    if(var.varValue==0):
      continue
    print(f"{var.name} = {var.varValue}")
print("Valor da função objetivo = ", value(model.objective))


Status: Optimal
x12 = 30.0
x13 = 30.0
x24 = 15.0
x31 = 20.0
x34 = 25.0
Valor da função objetivo =  3260.0


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


## Exercício 2

Refaça o modelo anterior, dessa vez impedindo a divisão de produtos entre fábricas. Ou seja, cada produto deve ser integralmente produzido em uma única fábrica, para eliminar custos ocultos associados à divisão da produção. A cada fábrica deve ser destinado pelo menos um produto.

### Resolução

In [61]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value, LpStatus

# Criação do modelo
model = LpProblem(name="minimization_problem", sense=LpMinimize)

# Adicionar variáveis
for i in range(1,4):
  for j in range(1,5):
    model += LpVariable(f"x{i}{j}", lowBound=0, cat='Integer')


# Adicionar variáveis
for i in range(1,4):
  for j in range(1,5):
    model += LpVariable(f"y{i}{j}", lowBound=0, cat='Integer')


# Definir a função objetivo
model += (41*x11 + 27*x12 + 28*x13 + 24*x14 +
          40*x21 + 29*x22 + 23*x24 +
          37*x31 + 30*x32 + 27*x33 + 21*x34), "Objective"

# Adicionar restrições
model += (x11 + x12 + x13 + x14 <= 75), "Constraint1"
model += (x21 + x22 + x24 <= 75), "Constraint2"
model += (x31 + x32 + x33 + x34 <= 45), "Constraint3"

model += x11 >= 20 * y11
model += x12 >= 30 * y12
model += x13 >= 30 * y13
model += x14 >= 40 * y14
model += x21 >= 20 * y21
model += x22 >= 30 * y22
model += x24 >= 40 * y24
model += x31 >= 20 * y31
model += x32 >= 30 * y32
model += x33 >= 30 * y33
model += x34 >= 40 * y34

model += y11 + y12 + y13 + y14 >= 1
model += y21 + y22 +  y24 >= 1
model += y31 + y32 + y33 + y34 >=1

model += y21 + y31 + y11 == 1
model += y22 + y12 +  y32 == 1
model += y13 + y33 ==1
model += y14 + y34 + y24 ==1


# Resolver o modelo
model.solve()

# Verificar e imprimir resultados
print("Status:", LpStatus[model.status])
for var in model.variables():
    if(var.name =="y11"):
      break
    if(var.varValue==0):
      continue
    print(f"{var.name} = {var.varValue}")
print("Valor da função objetivo = ", value(model.objective))


Status: Optimal
x12 = 30.0
x13 = 30.0
x21 = 20.0
x34 = 40.0
Valor da função objetivo =  3290.0
